In [27]:
#pip install requests beautifulsoup4

In [18]:
#pip install selenium
pip install webdriver-manager

SyntaxError: invalid syntax (4200850686.py, line 2)

In [20]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import random
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

In [2]:
def scrape_logo_urls(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "DNT": "1",
            "Referer": "https://ultiworld.com/ranking/",
        }
        
        # Add a session to maintain cookies
        session = requests.Session()
        response = session.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Add a small delay to avoid overwhelming the server
        time.sleep(2)
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Select all spans with the data-bg attribute within the specified structure
        spans = soup.select('body main div div.reference-ranking table.table tbody tr td.ranking__team span[data-bg]')
        
        # Initialize a list to store logo data
        logo_data = []
        
        for span in spans:
            # Extract the logo URL from the data-bg attribute
            logo_url = span.get('data-bg', None)
            
            # Extract the team name from the <a> tag within the span
            a_tag = span.find('a')
            team_name = a_tag.text.strip() if a_tag else 'Unknown Team'
            team_url = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None
            
            # Append the extracted data to the list
            logo_data.append({
                'team_name': team_name,
                'team_url': team_url,
                'logo_url': logo_url
            })
        
        # Convert the list of dictionaries to a DataFrame
        return pd.DataFrame(logo_data)

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred while scraping {url}: {http_err}")
    except requests.RequestException as req_err:
        print(f"Request error occurred while scraping {url}: {req_err}")
    except Exception as e:
        print(f"An unexpected error occurred while scraping {url}: {e}")
    
    return None

In [3]:
scrape_logo_urls('https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/')

HTTP error occurred while scraping https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/: 403 Client Error: Forbidden for url: https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/


In [4]:
def download_logos(df, folder_name='logos', delay=0.5):
    """
    Download images from URLs stored in a DataFrame
    """
    # Create logos directory in the current working directory
    logos_dir = os.path.join(os.getcwd(), folder_name)
    os.makedirs(logos_dir, exist_ok=True)
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Add a column for the local file path
    df['local_path'] = ''
    
    for idx, row in df.iterrows():
        try:
            # Extract the file extension from the URL
            file_extension = os.path.splitext(row['logo_url'])[1]
            
            # Generate filename using the team name
            filename = os.path.join(logos_dir, f"{row['team_name']}{file_extension}")
            
            # Skip downloading if the file already exists
            if os.path.exists(filename):
                print(f"File already exists, skipping download: {filename}")
                df.at[idx, 'local_path'] = filename
                continue
            
            # Download the image
            response = requests.get(row['logo_url'], headers=headers)
            response.raise_for_status()
            
            # Save the image
            with open(filename, 'wb') as f:
                f.write(response.content)
            
            # Store the local path in the DataFrame
            df.at[idx, 'local_path'] = filename
            print(f"Downloaded: {filename}")
            
            # Add a small delay
            time.sleep(delay)
            
        except requests.HTTPError as http_err:
            print(f"HTTP error occurred while downloading {row['logo_url']}: {http_err}")
        except requests.RequestException as req_err:
            print(f"Request error occurred while downloading {row['logo_url']}: {req_err}")
        except Exception as e:
            print(f"An unexpected error occurred while downloading {row['logo_url']}: {e}")
    
    return df

In [5]:
# # Define the URL
# url = 'https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/'

# # Scrape the logo URLs into a DataFrame
# logo_df = scrape_logo_urls(url)

# # Check if the DataFrame is not None and has data
# if logo_df is not None and not logo_df.empty:
#     print("Scraped data:")
#     print(logo_df.head())  # Display the first few rows to verify the data

#     # Download the logos
#     logo_df = download_logos(logo_df)

#     # Save the DataFrame with local paths to a CSV file
#     logo_df.to_csv('team_logos.csv', index=False)
#     print("Logos downloaded and data saved to 'team_logos.csv'.")
# else:
#     print("No data found to download.")

In [6]:
# Define the list of URLs
urls = [
    'https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/',
    'https://ultiworld.com/ranking/143018/college-d-i-mens-rankings-end-of-2023-season/',
    'https://ultiworld.com/ranking/119526/college-d-i-mens-rankings-end-of-2022-season/',
    'https://ultiworld.com/ranking/110899/college-d-i-mens-rankings-12-1-21/',
    'https://ultiworld.com/ranking/89868/college-d-i-mens-rankings-3-11-20/',
    'https://ultiworld.com/ranking/79575/college-d-i-mens-rankings-2019-season-final/',
    'https://ultiworld.com/ranking/66334/college-d-i-mens-rankings-2018-season-week-of-5-9-18/',
    'https://ultiworld.com/ranking/61700/college-d-i-mens-rankings-2018-season-preseason/',
    'https://ultiworld.com/ranking/50496/college-d-i-mens-rankings-2017-season-end-of-fall/',
    'https://ultiworld.com/ranking/38352/college-d-i-mens-rankings-2016-season-post-ccc/',
]

# Initialize an empty DataFrame to store all logo data
all_logo_data = pd.DataFrame()

# Loop through each URL
for url in urls:
    # Scrape the logo URLs into a DataFrame
    logo_df = scrape_logo_urls(url)

    # Check if the DataFrame is not None and has data
    if logo_df is not None and not logo_df.empty:
        print(f"Scraped data from {url}:")
        print(logo_df.head())  # Display the first few rows to verify the data

        # Append the data to the all_logo_data DataFrame
        all_logo_data = pd.concat([all_logo_data, logo_df], ignore_index=True)
    else:
        print(f"No data found to download from {url}.")

HTTP error occurred while scraping https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/: 403 Client Error: Forbidden for url: https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/
No data found to download from https://ultiworld.com/ranking/161780/college-d-i-mens-rankings-6-5-24/.
HTTP error occurred while scraping https://ultiworld.com/ranking/143018/college-d-i-mens-rankings-end-of-2023-season/: 403 Client Error: Forbidden for url: https://ultiworld.com/ranking/143018/college-d-i-mens-rankings-end-of-2023-season/
No data found to download from https://ultiworld.com/ranking/143018/college-d-i-mens-rankings-end-of-2023-season/.
HTTP error occurred while scraping https://ultiworld.com/ranking/119526/college-d-i-mens-rankings-end-of-2022-season/: 403 Client Error: Forbidden for url: https://ultiworld.com/ranking/119526/college-d-i-mens-rankings-end-of-2022-season/
No data found to download from https://ultiworld.com/ranking/119526/college-d-i-mens-rank

In [7]:
all_logo_data

""


In [8]:
# Check if all_logo_data is not empty
if not all_logo_data.empty:
    # Convert the team names to a list
    team_names = all_logo_data['team_name'].tolist()
    
    # Print each team name on a new line with indentation
    print("Team Names List:")
    for name in team_names:
        print(f"    {name}")
else:
    print("No team data available.")

No team data available.


In [9]:
# Change all instances of a team name manually

# Method 2: Replace with a dictionary of multiple changes
name_changes = {
    'Washington University': 'WashU',
    'Pennsylvania': 'Penn',
    'California-Davis': 'UC Davis',
    'California-Santa Cruz': 'UC Santa Cruz',
    'California-San Diego': 'UC San Diego',
    'California-Santa Barbara': 'UC Santa Barbara',
    'North Carolina-Wilmington': 'UNC Wilmington',
    'Texas-Dallas': 'UT Dallas'
}
# Apply the changes
all_logo_data['team_name'] = all_logo_data['team_name'].replace(name_changes)

# Verify the changes
all_logo_data['team_name'].value_counts()

KeyError: 'team_name'

In [22]:
# If there is any data, proceed to download
if not all_logo_data.empty:
    # Download the logos
    all_logo_data = download_logos(all_logo_data)

    # Save the DataFrame with local paths to a CSV file
    all_logo_data.to_csv('team_logos.csv', index=False)
    print("Logos downloaded and data saved to 'team_logos.csv'.")
else:
    print("No data found to download from any URL.")

File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Brown.jpeg
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Cal Poly SLO.jpg
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/North Carolina.png
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Georgia.jpeg
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Colorado.png
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Oregon.png
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Minnesota.png
File already exists, skipping download: /Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/logos/Massac

In [23]:
# Add data to the dataframe

# Method 3: Add multiple rows at once
new_data = {
    'team_name': ['Carnegie Mellon', 
                  'Team2',
                  'Team3'
                  ],

    'logo_url': ['https://cdn.ultiworld.com/wordpress/wp-content/uploads/2023/02/carnegie_mellon_logo-300x300.png', 
                 'URL2', 
                 'URL3'
                 ],
    # Add other columns as needed
}
new_rows = pd.DataFrame(new_data)
all_logo_data = pd.concat([all_logo_data, new_rows], ignore_index=True)

In [ ]:
########### Try again ##########

In [22]:
def scrape_logo_urls(url):
    try:
        # Set up Chrome options
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run in headless mode
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        
        # Initialize the driver
        driver = webdriver.Chrome(options=chrome_options)
        
        # Set page load timeout
        driver.set_page_load_timeout(20)
        
        try:
            # Navigate to the URL
            driver.get(url)
            
            # Wait for the table to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "td.ranking__team span[data-bg]"))
            )
            
            # Get the page source and create BeautifulSoup object
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Select all spans with the data-bg attribute within table cells
            spans = soup.select('td.ranking__team span[data-bg]')
            
            # Initialize a list to store logo data
            logo_data = []
            
            for span in spans:
                # Extract the logo URL from the data-bg attribute
                logo_url = span.get('data-bg', None)
                
                # Extract the team name and URL from the anchor tag
                a_tag = span.find('a')
                team_name = a_tag.text.strip() if a_tag else 'Unknown Team'
                team_url = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None
                
                # Extract the rank from the previous sibling td
                rank_td = span.find_parent('td').find_previous_sibling('td', class_='ranking__rank')
                rank = rank_td.text.strip() if rank_td else None
                
                # Append the extracted data to the list
                logo_data.append({
                    'rank': rank,
                    'team_name': team_name,
                    'team_url': team_url,
                    'logo_url': logo_url
                })
            
            # Convert the list of dictionaries to a DataFrame
            return pd.DataFrame(logo_data)
            
        finally:
            # Always close the driver
            driver.quit()

    except Exception as e:
        print(f"An error occurred while scraping {url}: {e}")
        return None

In [23]:
def download_logos(df, folder_name='logos', delay=0.5):
    """
    Download images from URLs stored in a DataFrame
    
    Parameters:
    df (DataFrame): DataFrame containing 'team_name' and 'logo_url' columns
    folder_name (str): Name of folder to store logos (default: 'logos')
    delay (float): Delay between downloads in seconds (default: 0.5)
    """
    # Create logos directory in the current working directory
    logos_dir = os.path.join(os.getcwd(), folder_name)
    os.makedirs(logos_dir, exist_ok=True)
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Add a column for the local file path
    df['local_path'] = ''
    
    for idx, row in df.iterrows():
        try:
            # Clean the team name to create a valid filename
            clean_team_name = "".join(c for c in row['team_name'] if c.isalnum() or c in (' ', '-')).strip()
            
            # Extract the file extension from the URL
            file_extension = os.path.splitext(row['logo_url'])[1]
            if not file_extension:  # If no extension found, default to .jpg
                file_extension = '.jpg'
            
            # Generate filename using the team name and rank
            filename = os.path.join(logos_dir, f"{row['rank']:02d}_{clean_team_name}{file_extension}")
            
            # Skip downloading if the file already exists
            if os.path.exists(filename):
                print(f"File already exists, skipping download: {filename}")
                df.at[idx, 'local_path'] = filename
                continue
            
            # Download the image
            response = requests.get(row['logo_url'], headers=headers)
            response.raise_for_status()
            
            # Save the image
            with open(filename, 'wb') as f:
                f.write(response.content)
            
            # Store the local path in the DataFrame
            df.at[idx, 'local_path'] = filename
            print(f"Downloaded: {filename}")
            
            # Add a small delay
            time.sleep(delay)
            
        except requests.HTTPError as http_err:
            print(f"HTTP error occurred while downloading {row['logo_url']}: {http_err}")
        except requests.RequestException as req_err:
            print(f"Request error occurred while downloading {row['logo_url']}: {req_err}")
        except Exception as e:
            print(f"An unexpected error occurred while downloading {row['logo_url']}: {e}")
    
    return df

In [24]:
# # Define the list of URLs
# urls = [
#     'https://ultiworld.com/ranking/171037/college-d-i-womens-preseason-rankings-1-17-25/',
#     'https://ultiworld.com/ranking/161779/college-d-i-womens-rankings-6-5-24/',
#     'https://ultiworld.com/ranking/143015/college-d-i-womens-rankings-end-of-2023-season/',
#     'https://ultiworld.com/ranking/119528/college-d-i-womens-rankings-end-of-2022-season/',
#     'https://ultiworld.com/ranking/110904/college-d-i-womens-rankings-12-1-21/',
#     'https://ultiworld.com/ranking/89870/college-d-i-womens-rankings-3-11-20/',
#     'https://ultiworld.com/ranking/79572/college-d-i-womens-rankings-2019-season-final/',
#     'https://ultiworld.com/ranking/66338/college-d-i-womens-rankings-2018-season-week-of-5-9-18/',
#     'https://ultiworld.com/ranking/61702/college-d-i-womens-rankings-2018-season-preseason/',
#     'https://ultiworld.com/ranking/50498/college-d-i-womens-rankings-2017-season-end-of-fall/',
#     'https://ultiworld.com/ranking/38358/college-d-i-womens-rankings-2016-season-post-ccc/',
#     'https://ultiworld.com/ranking/26614/college-d-i-womens-division-power-rankings-season-2015-11-10-14/'
# ]

# # Initialize an empty DataFrame to store all logo data
# all_logo_data = pd.DataFrame()

# # Loop through each URL
# for url in urls:
#     # Scrape the logo URLs into a DataFrame
#     logo_df = scrape_logo_urls(url)
    
#     # Check if the DataFrame is not None and has data
#     if logo_df is not None and not logo_df.empty:
#         print(f"Scraped data from {url}:")
#         print(logo_df.head())  # Display the first few rows to verify the data
        
#         # Append the data to the all_logo_data DataFrame
#         all_logo_data = pd.concat([all_logo_data, logo_df], ignore_index=True)
#     else:
#         print(f"No data found to download from {url}.")

An error occurred while scraping https://ultiworld.com/ranking/171037/college-d-i-womens-preseason-rankings-1-17-25/: Message: 
Stacktrace:
0   chromedriver                        0x0000000102ebe138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000102eb6988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x000000010291c968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000102960d4c cxxbridge1$string$len + 368752
4   chromedriver                        0x000000010299a4f0 cxxbridge1$string$len + 604180
5   chromedriver                        0x0000000102955564 cxxbridge1$string$len + 321672
6   chromedriver                        0x00000001029561b4 cxxbridge1$string$len + 324824
7   chromedriver                        0x0000000102e88fc0 cxxbridge1$str$ptr + 3436424
8   chromedriver                        0x0000000102e8c2dc cxxbridge1$str$ptr + 3449508
9   chromedriver                        0x0000000102e6fe60 

In [28]:
from bs4 import BeautifulSoup
import pandas as pd

def scrape_logo_urls_from_html(html_content):
    """Extract logo URLs and rankings from HTML content"""
    
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all team rows
    team_rows = soup.select('tr')
    
    teams = []
    for row in team_rows:
        # Get rank
        rank_cell = row.select_one('.ranking__rank')
        if not rank_cell:  # Skip rows without rank
            continue
            
        rank = int(rank_cell.text)
        
        # Get team info
        team_cell = row.select_one('.ranking__team')
        if not team_cell:  # Skip rows without team info
            continue
            
        team_name = team_cell.find('a').text
        logo_span = team_cell.select_one('.snippet-inline')
        logo_url = logo_span.get('data-bg', '') if logo_span else ''
        
        teams.append({
            'rank': rank,
            'team': team_name,
            'logo_url': logo_url
        })
    
    return pd.DataFrame(teams)

NameError: name 'html_content' is not defined

In [29]:
# Use the function with your HTML file
html_file_path = '/Users/elistandard/Documents/CS_Projects/Github/ultimateDynasties/college-womens/logo-html-files/2025-womens-ultiworld-logos.rtf'  # Replace with your file path
df = scrape_logo_urls_from_file(html_file_path)

print(df)

# Make sure the HTML content is passed as a string
# You can verify the content type:
print(type(html_content))  # Should be <class 'str'>

# Use the function
df = scrape_logo_urls_from_html(html_content)
print(df)

Empty DataFrame
Columns: []
Index: []


/var/folders/mm/34qgr9ns2gd47_6dllf0zrv80000gn/T/ipykernel_4544/3117128758.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')


NameError: name 'html_content' is not defined

In [27]:
print("First 100 characters of HTML:")
print(html_content[:100])

First 100 characters of HTML:


NameError: name 'html_content' is not defined